# Primary analyses of our sequences

In [1]:
import os
import sys
import pandas as pd
import qiime2 as q2
from qiime2 import Visualization
import matplotlib.pyplot as plt
%matplotlib inline

# Define the data directory
data_dir = '/home/jovyan/FunGut2/full-pipeline/data'

In [2]:
import os

In [4]:
!pwd

/home/jovyan/FunGut2/Nicola


In [20]:
!qiime sample-classifier classify-samples --help

Usage: qiime sample-classifier classify-samples [OPTIONS]

  Predicts a categorical sample metadata column using a supervised learning
  classifier. Splits input data into training and test sets. The training set
  is used to train and test the estimator using a stratified k-fold cross-
  validation scheme. This includes optional steps for automated feature
  extraction and hyperparameter optimization. The test set validates
  classification accuracy of the optimized estimator. Outputs classification
  results for test set. For more details on the learning algorithm, see
  http://scikit-learn.org/stable/supervised_learning.html

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency | RelativeFrequency |
    PresenceAbsence | Composition]
                          Feature table containing all features that should
                          be used for target prediction.            [required]
Parameters:
  --m-metadata-file METADATA
  --m-metadata-column COLUMN  MetadataColumn[Categorical]


In [8]:
!pwd

/home/jovyan/FunGut2/Nicola


In [3]:
! rm -r machLearning

In [4]:
!qiime sample-classifier classify-samples \
    --i-table $data_dir/filtered-feature-table.qza \
    --m-metadata-file $data_dir/metadata/fungut_metadata_processed.tsv \
    --m-metadata-column is_urban \
    --p-random-state 42 \
    --p-parameter-tuning \
    --p-estimator KNeighborsClassifier \
    --output-dir ./machLearning/

Saved SampleEstimator[Classifier] to: ./machLearning/sample_estimator.qza
Saved FeatureData[Importance] to: ./machLearning/feature_importance.qza
Saved SampleData[ClassifierPredictions] to: ./machLearning/predictions.qza
Saved Visualization to: ./machLearning/model_summary.qzv
Saved Visualization to: ./machLearning/accuracy_results.qzv
Saved SampleData[Probabilities] to: ./machLearning/probabilities.qza
Saved Visualization to: ./machLearning/heatmap.qzv
Saved SampleData[TrueTargets] to: ./machLearning/training_targets.qza
Saved SampleData[TrueTargets] to: ./machLearning/test_targets.qza


In [5]:
Visualization.load('./machLearning/accuracy_results.qzv')

<visualization: Visualization uuid: 44a52309-1786-45a7-8f32-63bc6d76a4d0>